# Download YouTube Video's Audio

In [1]:
! pip install pytube -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00


In [2]:
from pytube import YouTube

In [69]:
#VIDEO_URL = "https://youtu.be/O-kxuJtlYSg?si=jqnzwJ4hr7PEG0YP" #Diksha

In [96]:
VIDEO_URL = 'https://youtu.be/O8ZXR9bt_90?si=BdR-7hgHsfA03cKQ' #batman

In [5]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [97]:
yt = YouTube(VIDEO_URL)

In [98]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4')

'/content/ytaudio.mp4'

In [99]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# English ASR with HuggingSound

In [100]:
!pip install huggingsound -q

In [101]:
from huggingsound import SpeechRecognitionModel

In [102]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [103]:
device

'cuda'

In [104]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


# Audio Chunking

In [105]:
import librosa

In [106]:
input_file = '/content/ytaudio.wav'

In [107]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [108]:
import soundfile as sf


In [109]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [110]:
i

17

# Audio Transcription / ASR / Speech to Text

In [111]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [112]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav',
 '/content/10.wav',
 '/content/11.wav',
 '/content/12.wav',
 '/content/13.wav',
 '/content/14.wav',
 '/content/15.wav',
 '/content/16.wav',
 '/content/17.wav']

In [113]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 18/18 [00:10<00:00,  1.66it/s]


In [114]:
full_transcript = ' '

In [115]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [116]:
len(full_transcript)

7110

In [117]:
full_transcript

" ergreat news i've just heard the best newsi just went on twitter-dot net and it turns out according to many a music fan over there i am sexiust i'm not even rn had no idea iwas being such a sexi reviewboy sexius reviewer without even going for it is just like such an incredible accalaid that didn't didn't reallygenerallyooosxareyou sureohturns out i'm not the sexiust reviewer turns out i'm a sexus revieweryes online the discourse is discoursing once again ataking shape in very interesting ways around the reviewsmost recently when it's come to takesave had around a couple of recent albums this year on the new taylor-swift lp the tortured poets department and two the recent gracy abrums album the secret of usone who due to her songwriting style and the taylor-swift feature packed into that record i referred to as kind of taylorswift's baby creativeagain her songs are very derivative of miss swift's tunes her vocals so pretty much lord ripoffonce again haven't really heard somebody out 

# Text Summarization

In [118]:
from transformers import pipeline

In [119]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Text Chunking before Summarization

In [122]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

#print(summarized_text)

In [123]:
summarized_text

[' The discourse is discoursing once again ataking shape in very interesting ways around the',
 ' The production on her latest record is packed with producers and collaborators that taylors',
 " The point of this video isn't to laugh off the idea of sexism especially when it",
 ' Taylor Swift is at a peak of her career commercially n financially and selling out tours and',
 " Taylor Swift's fans defend her against all sexus criticism being thrown at her . Singer",
 ' The discourse around many a taylor-swift twitter account on the internet often these',
 ' The non-sexist thg to do would hold that artist to a level of',
 ' anthony fantano  the sexiistforever. he channel which is on']